In [1]:
import pandas as pd
import os
import zipfile
import json
import re
from transformers import pipeline

c:\Users\justi\anaconda3\envs\capstone\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
# zip_path = r"C:\Users\justi\.cache\modelscope\hub\datasets\JimmyMa99\TeleAntiFraud-28k\TeleAntiFraud-28k.zip"
# extract_path = r"../data/"

# with zipfile.ZipFile(zip_path, 'r') as zip_ref:
#     zip_ref.extractall(extract_path)

# print("Extracted to:", extract_path)

Extracted to: ../data/


In [ ]:
# base_dir = r"../data/TeleAntiFraud-28k/merged_result"
# rows = []

# for root, dirs, files in os.walk(base_dir):
#     for file in files:
#         if file == "config.json":
#             with open(os.path.join(root, file), "r") as f:
#                 data = json.load(f)
                
#             # Aggregate left/right messages
#             left_texts = [seg["content"] for seg in data["audio_segments"] if seg["role"] == "left"]
#             right_texts = [seg["content"] for seg in data["audio_segments"] if seg["role"] == "right"]

#             # Combine full conversation (in chronological order)
#             full_text = " ".join([seg["content"] for seg in data["audio_segments"]])

#             # Determine label (POS/NEG)
#             label = "POS" if "POS" in root else "NEG"

#             rows.append({
#                 "conversation_id": root.split(os.sep)[-1],
#                 "label": label,
#                 "left_text": " ".join(left_texts),
#                 "right_text": " ".join(right_texts),
#                 "full_text": full_text
#             })

# df = pd.DataFrame(rows)
# df.to_csv("../data/TeleAntiFraud/cleaned_conversations.csv", index=False, encoding="utf-8-sig")
# print(f"✅ Extracted {len(df)} conversations.")


✅ Extracted 9487 conversations.


In [ ]:
def detect_fraud_type(text):
    if pd.isna(text) or text.strip() == "":
        return None

    text = text.lower()

    # Define keyword patterns using chinese characters
    fraud_patterns = {
        "Investment Scam": r"(投资|理财|收益|基金|股票|回报|分红)",
        "Loan Scam": r"(贷款|借款|信用|征信|额度|分期)",
        "Telecom Scam": r"(电话|公安|银行|账户|冻结|验证码|转账)",
        "Impersonation Scam": r"(警察|客服|工作人员|冒充|身份)",
        "Lottery/Prize Scam": r"(中奖|礼品|领取|抽奖|奖励|活动)"
    }

    for fraud_type, pattern in fraud_patterns.items():
        if re.search(pattern, text):
            return fraud_type

    return "Other Scam"  # default for fraud but unknown type

In [17]:
df = pd.read_csv("../data/TeleAntiFraud/cleaned_conversations.csv")
df["fraud_type"] = df.apply(
    lambda row: detect_fraud_type(row["full_text"]) if row["label"] == "NEG" else None,
    axis=1
)

In [18]:
df

,conversation_id,label,left_text,right_text,full_text,fraud_type
0,tts_test1,NEG,喂，你好 我是来自金融投资咨询中心的小李，我们注意到你最近在寻找投资机会。 太好了，我们有一...,嗯，你好 哦，是吗，我确实有点兴趣。 嗯，听起来不错，具体是什么项目呢？ 数字货币啊，听起来...,喂，你好 嗯，你好 我是来自金融投资咨询中心的小李，我们注意到你最近在寻找投资机会。 哦，是...,Investment Scam
1,tts_test10,NEG,喂，你好，我是中行渠道部的小李。 我们这边有一个特别优惠的贷款产品，利率特别低，只有两厘，你...,你好，有什么事情吗？ 可以啊，这个贷款怎么申请？ 下载APP？我怕下载了以后有问题。 那我需...,喂，你好，我是中行渠道部的小李。 你好，有什么事情吗？ 我们这边有一个特别优惠的贷款产品，利...,Loan Scam
2,tts_test100,NEG,喂，你好，这边是全国快递客服专线，我是客服专员小李。 你的一个快递包裹在运输途中不慎丢失，我...,你好，有什么事吗？ 真的吗？那太感谢你了。 好的，我的银行卡号是六二二八四八零零零零零零零零...,喂，你好，这边是全国快递客服专线，我是客服专员小李。 你好，有什么事吗？ 你的一个快递包裹在...,Telecom Scam
3,tts_test1000,NEG,喂，你好，我是农商银行的张经理。 根据你名下的华瑞精密模具厂的营业执照，我们银行可以为你提供...,你好，张经理。 啊，这个暂时不需要。 嗯，我再考虑考虑。 嗯，好的。 好的，我知道了。,喂，你好，我是农商银行的张经理。 你好，张经理。 根据你名下的华瑞精密模具厂的营业执照，我们...,Loan Scam
4,tts_test1001,NEG,你好，请问是张华吗？ 我是京东金融的客服专员，我们注意到你之前有浏览过我们的贷款产品，现在我...,是我，你是谁啊？ 贷款？我现在手头还行，暂时不考虑。 那听起来不错，但我在其他银行也有贷款，...,你好，请问是张华吗？ 是我，你是谁啊？ 我是京东金融的客服专员，我们注意到你之前有浏览过我们...,Loan Scam
...,...,...,...,...,...,...
9482,tts_test2996,POS,喂，你好 我想咨询一下你们餐厅的菜单，还有营业时间。 那你们有什么推荐的菜品吗？ 听上去不错...,你好，有什么可以帮助你的吗？ 当然可以，我们餐厅提供中式和西式菜品，营业时间是从早上十点到晚...,喂，你好 你好，有什么可以帮助你的吗？ 我想咨询一下你们餐厅的菜单，还有营业时间。 当然可以...,None
9483,tts_test2997,POS,喂 嗯 啊，怎么会这样呢？ 那我怎么办，要不我再送一份给你？ 那真是太谢谢你了，下次一定注意...,喂喂，你好，我是美团骑手 就是你这个砂锅，汤洒了一大半，还能吃吗？ 可能是路上颠簸得太厉害，...,喂 喂喂，你好，我是美团骑手 嗯 就是你这个砂锅，汤洒了一大半，还能吃吗？ 啊，怎么会这样呢...,None
9484,tts_test2998,POS,喂，你好，我是小明，最近咱们平台有活动，你有空来逛逛吗？ 有有有，咱们平台新上了不少电子产品...,你好，小明，最近挺忙的，有什么好东西推荐吗？ 那挺好的，我最近正好想换个手机，有什么推荐的吗...,喂，你好，我是小明，最近咱们平台有活动，你有空来逛逛吗？ 你好，小明，最近挺忙的，有什么好东...,None
9485,tts_test2999,POS,喂，你好，我是小区的保安，你刚才打电话说要订餐是吗？ 是这样的，我们小区有个规定，晚上十二点...,对，我刚才确实订了餐，但是他们说不能送到我这栋楼，怎么回事？ 那我住在十九栋幺零六室，现在躺...,喂，你好，我是小区的保安，你刚才打电话说要订餐是吗？ 对，我刚才确实订了餐，但是他们说不能送...,None


In [21]:
from deep_translator import GoogleTranslator # could also use other translators like deepl or openai, but google translator is the fastest
from tqdm import tqdm

translator = GoogleTranslator(source='zh-CN', target='en')

df["perp_text_en"] = ""
# df["victim_text_en"] = ""
# df["full_text_en"] = ""

for i in tqdm(range(len(df))):
    try:
        if df.loc[i, "label"] == "NEG": # could also use df.loc[i, "fraud_type"] == "Loan Scam" for just translating loan scam cases or comment out for all cases
            df.loc[i, "perp_text_en"] = translator.translate(df.loc[i, "left_text"]) if pd.notna(df.loc[i, "left_text"]) else ""
            # df.loc[i, "victim_text_en"] = translator.translate(df.loc[i, "right_text"]) if pd.notna(df.loc[i, "right_text"]) else ""
            # df.loc[i, "full_text_en"] = translator.translate(df.loc[i, "full_text"]) if pd.notna(df.loc[i, "full_text"]) else ""
    except Exception as e:
        print(f"Error at row {i}: {e}")
        continue


 32%|███▏      | 3029/9487 [41:26<1:28:21,  1.22it/s]


KeyboardInterrupt: 

In [22]:
df.to_csv("../data/TeleAntiFraud/cleaned_conversations.csv", index=False, encoding="utf-8-sig")